In [1]:
from __future__ import print_function, division

from keras.datasets import fashion_mnist,cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Conv2DTranspose
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers import Concatenate, GaussianNoise,Activation
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(2 * 2 * 512, activation='relu',input_dim=self.latent_dim))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))
        model.add(Reshape((2, 2, 512)))

        model.add(Conv2DTranspose(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))


        model.add(Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))

        model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
        model.add( BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))


        model.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)
    def build_discriminator(self):
        model = Sequential()

        # Conv 1: 16x16x64
        model.add(Conv2D(64, kernel_size=5, strides=2, padding='same' ,input_shape=self.img_shape))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))

        # Conv 2:
        model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))

        # Conv 3:
        model.add(Conv2D(256, kernel_size=5, strides=2, padding='same'))
        model.add( BatchNormalization(momentum=0.9))
        model.add( LeakyReLU(alpha=0.1))

        # Conv 4:
        model.add(Conv2D(512, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add( LeakyReLU(alpha=0.1))
        model.summary()

        # FC
        model.add(Flatten())
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')
        labels = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))
        discriminator =model(img)

        # Concatenate
        merged_layer = Concatenate()([discriminator, labels])
        discriminator = Dense(512, activation='relu')(merged_layer)

        # Output
        discriminator = Dense(1, activation='sigmoid')(discriminator)

        return Model([img, label], discriminator)




    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = cifar10.load_data()

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                labels = y_train[index * batch_size:(index + 1) * batch_size]

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Condition on labels
                sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if  global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)


    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        generated_images = self.generator.predict([noise, sampled_labels])


        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

        def vis_square(data, padsize=1, padval=0):
            # force the number of filters to be square
            n = int(np.ceil(np.sqrt(data.shape[0])))
            padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
            data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

            # tile the filters into an image
            data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
            data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
            return data

        img = vis_square(generated_images)
        if not os.path.isdir('images_cgan_cifar10'):
            os.mkdir('images_cgan_cifar10')
        Image.fromarray(img).save(
            "images_cgan_cifar10/epoch_%d_step_%d.png" % (epoch, global_step))


if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 128)         512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 256)         819456    
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 3.709824, acc.: 30.47%] [G loss: 0.363816]
epoch:0 step:2 [D loss: 0.877001, acc.: 50.00%] [G loss: 3.849415]
epoch:0 step:3 [D loss: 0.933845, acc.: 53.91%] [G loss: 2.861706]
epoch:0 step:4 [D loss: 0.786779, acc.: 42.97%] [G loss: 3.696392]
epoch:0 step:5 [D loss: 0.290398, acc.: 92.19%] [G loss: 3.537023]
epoch:0 step:6 [D loss: 0.957640, acc.: 45.31%] [G loss: 4.715917]
epoch:0 step:7 [D loss: 0.332690, acc.: 83.59%] [G loss: 3.808960]
epoch:0 step:8 [D loss: 0.319941, acc.: 96.88%] [G loss: 4.356854]
epoch:0 step:9 [D loss: 0.089602, acc.: 99.22%] [G loss: 4.339054]
epoch:0 step:10 [D loss: 0.672593, acc.: 50.78%] [G loss: 7.162156]
epoch:0 step:11 [D loss: 0.255331, acc.: 85.94%] [G loss: 6.342080]
epoch:0 step:12 [D loss: 0.137135, acc.: 95.31%] [G loss: 4.453283]
epoch:0 step:13 [D loss: 0.086170, acc.: 100.00%] [G loss: 4.860901]
epoch:0 step:14 [D loss: 0.033759, acc.: 100.00%] [G loss: 5.045748]
epoch:0 step:15 [D loss: 0.386862, acc.: 71.09%] [G los

epoch:0 step:121 [D loss: 0.368158, acc.: 81.25%] [G loss: 6.699224]
epoch:0 step:122 [D loss: 0.172715, acc.: 94.53%] [G loss: 5.832705]
epoch:0 step:123 [D loss: 0.237150, acc.: 92.19%] [G loss: 4.416577]
epoch:0 step:124 [D loss: 0.637341, acc.: 82.03%] [G loss: 6.684715]
epoch:0 step:125 [D loss: 0.546957, acc.: 77.34%] [G loss: 3.846003]
epoch:0 step:126 [D loss: 0.088042, acc.: 100.00%] [G loss: 3.445774]
epoch:0 step:127 [D loss: 0.055817, acc.: 98.44%] [G loss: 2.409361]
epoch:0 step:128 [D loss: 0.181096, acc.: 95.31%] [G loss: 2.810153]
epoch:0 step:129 [D loss: 0.106473, acc.: 97.66%] [G loss: 3.590302]
epoch:0 step:130 [D loss: 0.133945, acc.: 97.66%] [G loss: 3.101332]
epoch:0 step:131 [D loss: 0.101992, acc.: 97.66%] [G loss: 1.488558]
epoch:0 step:132 [D loss: 0.242752, acc.: 92.19%] [G loss: 6.181052]
epoch:0 step:133 [D loss: 0.531168, acc.: 72.66%] [G loss: 0.419150]
epoch:0 step:134 [D loss: 0.182424, acc.: 94.53%] [G loss: 3.003994]
epoch:0 step:135 [D loss: 0.01508

...

epoch:49 step:38521 [D loss: 0.000189, acc.: 100.00%] [G loss: 0.000637]
epoch:49 step:38522 [D loss: 0.000471, acc.: 100.00%] [G loss: 0.000393]
epoch:49 step:38523 [D loss: 0.000025, acc.: 100.00%] [G loss: 0.000046]
epoch:49 step:38524 [D loss: 0.000061, acc.: 100.00%] [G loss: 0.000356]
epoch:49 step:38525 [D loss: 0.000021, acc.: 100.00%] [G loss: 0.000636]
epoch:49 step:38526 [D loss: 0.000195, acc.: 100.00%] [G loss: 0.000027]
epoch:49 step:38527 [D loss: 0.000581, acc.: 100.00%] [G loss: 0.004753]
epoch:49 step:38528 [D loss: 0.000018, acc.: 100.00%] [G loss: 0.000128]
epoch:49 step:38529 [D loss: 0.000101, acc.: 100.00%] [G loss: 0.000703]
epoch:49 step:38530 [D loss: 0.001949, acc.: 100.00%] [G loss: 0.000078]
epoch:49 step:38531 [D loss: 0.000037, acc.: 100.00%] [G loss: 0.000071]
epoch:49 step:38532 [D loss: 0.000119, acc.: 100.00%] [G loss: 0.003344]
epoch:49 step:38533 [D loss: 0.000068, acc.: 100.00%] [G loss: 0.000208]
epoch:49 step:38534 [D loss: 0.000200, acc.: 100.00

epoch:49 step:38637 [D loss: 0.000006, acc.: 100.00%] [G loss: 0.000019]
epoch:49 step:38638 [D loss: 0.000156, acc.: 100.00%] [G loss: 0.000039]
epoch:49 step:38639 [D loss: 0.000063, acc.: 100.00%] [G loss: 0.000034]
epoch:49 step:38640 [D loss: 0.000435, acc.: 100.00%] [G loss: 0.000018]
epoch:49 step:38641 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.000220]
epoch:49 step:38642 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.000031]
epoch:49 step:38643 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.001048]
epoch:49 step:38644 [D loss: 0.000019, acc.: 100.00%] [G loss: 0.000040]
epoch:49 step:38645 [D loss: 0.000024, acc.: 100.00%] [G loss: 0.000021]
epoch:49 step:38646 [D loss: 0.000039, acc.: 100.00%] [G loss: 0.000144]
epoch:49 step:38647 [D loss: 0.004612, acc.: 100.00%] [G loss: 0.007971]
epoch:49 step:38648 [D loss: 0.001252, acc.: 100.00%] [G loss: 0.000083]
epoch:49 step:38649 [D loss: 0.000256, acc.: 100.00%] [G loss: 0.000081]
epoch:49 step:38650 [D loss: 0.001044, acc.: 100.00

epoch:49 step:38753 [D loss: 0.000073, acc.: 100.00%] [G loss: 0.001275]
epoch:49 step:38754 [D loss: 0.000041, acc.: 100.00%] [G loss: 0.000522]
epoch:49 step:38755 [D loss: 0.000221, acc.: 100.00%] [G loss: 0.000648]
epoch:49 step:38756 [D loss: 0.000201, acc.: 100.00%] [G loss: 0.000174]
epoch:49 step:38757 [D loss: 0.000029, acc.: 100.00%] [G loss: 0.001605]
epoch:49 step:38758 [D loss: 0.000556, acc.: 100.00%] [G loss: 0.000772]
epoch:49 step:38759 [D loss: 0.000245, acc.: 100.00%] [G loss: 0.000548]
epoch:49 step:38760 [D loss: 0.000029, acc.: 100.00%] [G loss: 0.000189]
epoch:49 step:38761 [D loss: 0.000131, acc.: 100.00%] [G loss: 0.000292]
epoch:49 step:38762 [D loss: 0.000060, acc.: 100.00%] [G loss: 0.000226]
epoch:49 step:38763 [D loss: 0.000002, acc.: 100.00%] [G loss: 0.000754]
epoch:49 step:38764 [D loss: 0.000075, acc.: 100.00%] [G loss: 0.000443]
epoch:49 step:38765 [D loss: 0.000010, acc.: 100.00%] [G loss: 0.000355]
epoch:49 step:38766 [D loss: 0.000015, acc.: 100.00

epoch:49 step:38869 [D loss: 0.001509, acc.: 100.00%] [G loss: 0.001300]
epoch:49 step:38870 [D loss: 0.004536, acc.: 100.00%] [G loss: 0.000820]
epoch:49 step:38871 [D loss: 0.013978, acc.: 100.00%] [G loss: 0.066206]
epoch:49 step:38872 [D loss: 0.000050, acc.: 100.00%] [G loss: 0.245754]
epoch:49 step:38873 [D loss: 0.014051, acc.: 100.00%] [G loss: 0.032070]
epoch:49 step:38874 [D loss: 0.001442, acc.: 100.00%] [G loss: 0.000476]
epoch:49 step:38875 [D loss: 0.000288, acc.: 100.00%] [G loss: 0.000560]
epoch:49 step:38876 [D loss: 0.000019, acc.: 100.00%] [G loss: 0.057442]
epoch:49 step:38877 [D loss: 0.000002, acc.: 100.00%] [G loss: 0.003446]
epoch:49 step:38878 [D loss: 0.000301, acc.: 100.00%] [G loss: 0.030849]
epoch:49 step:38879 [D loss: 0.000019, acc.: 100.00%] [G loss: 0.000936]
epoch:49 step:38880 [D loss: 0.000091, acc.: 100.00%] [G loss: 0.001335]
epoch:49 step:38881 [D loss: 0.000012, acc.: 100.00%] [G loss: 0.000374]
epoch:49 step:38882 [D loss: 0.000286, acc.: 100.00

epoch:49 step:38985 [D loss: 0.020560, acc.: 99.22%] [G loss: 0.001927]
epoch:49 step:38986 [D loss: 0.025735, acc.: 100.00%] [G loss: 0.000137]
epoch:49 step:38987 [D loss: 0.001449, acc.: 100.00%] [G loss: 0.002807]
epoch:49 step:38988 [D loss: 0.000594, acc.: 100.00%] [G loss: 0.000302]
epoch:49 step:38989 [D loss: 0.043789, acc.: 99.22%] [G loss: 0.000067]
epoch:49 step:38990 [D loss: 0.000654, acc.: 100.00%] [G loss: 0.000023]
epoch:49 step:38991 [D loss: 0.001069, acc.: 100.00%] [G loss: 0.000007]
epoch:49 step:38992 [D loss: 0.000537, acc.: 100.00%] [G loss: 0.000008]
epoch:49 step:38993 [D loss: 0.105021, acc.: 96.09%] [G loss: 0.143792]
epoch:49 step:38994 [D loss: 0.010640, acc.: 100.00%] [G loss: 0.833050]
epoch:49 step:38995 [D loss: 0.010526, acc.: 99.22%] [G loss: 0.120712]
epoch:49 step:38996 [D loss: 0.025748, acc.: 98.44%] [G loss: 0.044934]
epoch:49 step:38997 [D loss: 0.007924, acc.: 100.00%] [G loss: 1.031363]
epoch:49 step:38998 [D loss: 0.000184, acc.: 100.00%] [G